# 1. Formulação do problema

    Variáveis de Decisão:
        ti​ - Tempo de pouso do avião i (variável contínua);
        xij​ - Variável binária que vale 1 se o avião i pousar antes do avião j, 0 caso contrário;
        ei - Adiantamento do tempo ideal de pouso para o avião i (ei ≥ 0);
        di - Atraso do tempo ideal de pouso para o avião i (di ≥ 0).

    Parâmetros:
    
        Ri - tempo de sua detecção pelo radar;
        Ei - tempo inicial de pouso;
        Ti - tempo ideal para o pouso;
        Li - tempo final que o avião i pode pousar;
        
    Função Objetivo:

        Minimizar a penalidade total de pousos fora do tempo ideal:

## **Minimizar ∑ (gi * ei + hi * di)**

Onde:

    gi​ e hi​ são as penalidades por pousar antes ou depois do tempo ideal, respectivamente.

    S.a:
        
        Restrições de Viabilidade:
        
            Cada avião deve pousar dentro do seu intervalo permitido:

                Ei ​≤ ti ​≤ Li ​∀i 

        Restrições de Separação:
            O intervalo entre os pousos deve ser suficiente para garantir segurança:

                tj ≥ ti + sij - M(1 - xij) ∀i,j, i≠j

                M representa um valor grande.
        
        Variáveis Binárias:

                xij​ + xji ​= 1 ∀i, j, i≠j

        Cálculo de adiantamento e atraso:
                ei ​≥ Ti ​− ti  ​∀i
                di ≥ ti − Ti  ∀i
                



# Resolvendo usando o solver GLPK

In [54]:
import time
from pulp import value, LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, LpStatus, GLPK_CMD

def ler_entrada_do_arquivo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        linha_inicial = arquivo.readline().strip().split()
        qtdeDeAvioes = int(linha_inicial[0])

        tempos = []
        separacoes = []
        penalidades = []

        for _ in range(qtdeDeAvioes):
            linha_tempos = arquivo.readline().strip().split()
            while len(linha_tempos) < 6:
                linha_tempos.extend(arquivo.readline().strip().split())
            ri, ei, ti, li, gi, hi = map(float, linha_tempos[:6])
            tempos.append((ri, ei, ti, li, gi, hi))
            penalidades.append((gi, hi))

            linha_separacao = []
            #Para adequar arquivos .dat com erro na formatacao
            while len(linha_separacao) < qtdeDeAvioes:
                linha_separacao.extend(arquivo.readline().strip().split())
            separacoes.append(list(map(float, linha_separacao[:qtdeDeAvioes])))

        return qtdeDeAvioes, tempos, separacoes, penalidades


def solverSequenciamento(n, tempos, separacoes, penalidades, instance_name):
    E = [temp[0] for temp in tempos]
    T = [temp[1] for temp in tempos]
    L = [temp[2] for temp in tempos]
    g = [penalidade[0] for penalidade in penalidades]
    h = [penalidade[1] for penalidade in penalidades]

    modelo = LpProblem("SequenciamentoPousos", LpMinimize)

    #Variaveis de decisao 
    t = [LpVariable(f"t_{i}", lowBound=E[i], upBound=L[i], cat='Continuous') for i in range(n)]
    e = [LpVariable(f"e_{i}", lowBound=0, cat='Continuous') for i in range(n)]
    d = [LpVariable(f"d_{i}", lowBound=0, cat='Continuous') for i in range(n)]
    x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) if i != j else None for j in range(n)] for i in range(n)]

    #Funcao Objetivo
    modelo += lpSum(g[i] * e[i] + h[i] * d[i] for i in range(n)), "PenalidadeTotal"

    #Restricoes de separacao e sequenciamento
    M = max(L) + max(separacoes[i][j] for i in range(n) for j in range(n) if i != j)

    for i in range(n):
        for j in range(n):
            if i != j:
                modelo += t[j] >= t[i] + separacoes[i][j] - M * (1 - x[i][j]), f"Separacao_{i}_{j}"
                modelo += x[i][j] + x[j][i] == 1, f"Sequencia_{i}_{j}"

    #Calculo de adiantamento e atraso
    for i in range(n):
        modelo += e[i] >= T[i] - t[i], f"Adiantamento_{i}"
        modelo += d[i] >= t[i] - T[i], f"Atraso_{i}"

    #Resolver o modelo  
    inicioTempo = time.time()  
    modelo.solve(GLPK_CMD(msg=True, timeLimit=300)) #talvez o tempo esteja impedindo as respostas do 06/07/08.dat
    tempoExecucao = time.time() - inicioTempo

    #Escreve as respostas solicitadas no arquivo .txt
    with open("solucoesOtimas.txt", "a") as arquivo_resultado:
        arquivo_resultado.write(f"\nInstância: {instance_name}\n")
        if LpStatus[modelo.status] == "Optimal":
            temposPouso = [t[i].varValue for i in range(n)]
            valor_otimo = value(modelo.objective)
            arquivo_resultado.write(f"Valor ótimo da solução (Solver): {valor_otimo}\n")
            arquivo_resultado.write(f"Tempos de pouso: {', '.join(map(str, temposPouso))}\n")
            arquivo_resultado.write(f"Tempo de execução do Solver: {tempoExecucao:.2f} segundos\n")
        else:
            arquivo_resultado.write("Não encontrou solução ótima.\n")


def executarTodasAsInstancias():
    for i in range(1, 9):
        caminho_arquivo = f"problema_do_aviao/instances/0{i}.dat"
        qtdeDeAvioes, tempos, separacoes, penalidades = ler_entrada_do_arquivo(caminho_arquivo)
        solverSequenciamento(qtdeDeAvioes, tempos, separacoes, penalidades, f"0{i}.dat")

executarTodasAsInstancias()


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/28889f41fb7f46adb9fc521933fb5fa4-pulp.lp -o /tmp/28889f41fb7f46adb9fc521933fb5fa4-pulp.sol
 --tmlim 300
Reading problem data from '/tmp/28889f41fb7f46adb9fc521933fb5fa4-pulp.lp'...
200 rows, 120 columns, 490 non-zeros
90 integer variables, all of which are binary
309 lines were read
GLPK Integer Optimizer 5.0
200 rows, 120 columns, 490 non-zeros
90 integer variables, all of which are binary
Preprocessing...
84 constraint coefficient(s) were reduced
193 rows, 114 columns, 470 non-zeros
84 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.280e+02  ratio =  2.280e+02
GM: min|aij| =  6.880e-01  max|aij| =  1.454e+00  ratio =  2.113e+00
EQ: min|aij| =  4.844e-01  max|aij| =  1.000e+00  ratio =  2.064e+00
2N: min|aij| =  2.500e-01  max|aij| =  1.703e+00  ratio =  6.812e+00
Constructing initial basis...
Size of triangular part is 151
Solving LP relaxation...
GL

KeyboardInterrupt: 

+ 28800: mip =     not found yet >=   4.950000000e+02        (5996; 152)
+1216805: mip =   2.294000000e+04 >=   1.070000000e+03  95.3% (259648; 11025)
+ 29680: mip =     not found yet >=   4.950000000e+02        (6188; 156)
+1217525: mip =   2.294000000e+04 >=   1.070000000e+03  95.3% (259800; 11028)
+ 30285: mip =     not found yet >=   4.950000000e+02        (6326; 159)
+1218395: mip =   2.294000000e+04 >=   1.070000000e+03  95.3% (259961; 11032)
+ 31197: mip =     not found yet >=   4.950000000e+02        (6513; 163)
+1219176: mip =   2.294000000e+04 >=   1.070000000e+03  95.3% (260125; 11034)
+ 32133: mip =     not found yet >=   4.950000000e+02        (6683; 169)
+1220014: mip =   2.294000000e+04 >=   1.070000000e+03  95.3% (260302; 11038)
+ 32919: mip =     not found yet >=   4.950000000e+02        (6885; 171)
+1220671: mip =   2.294000000e+04 >=   1.070000000e+03  95.3% (260430; 11041)
+ 33161: mip =     not found yet >=   5.100000000e+02        (6960; 173)
+1221268: mip =   2.2